In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
from sklearn.compose import make_column_transformer

In [10]:
currency_pairs = ['USDCHF10080',
                 'GBPUSD10080', 'EURUSD10080', 'USDJPY10080', 'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 'GBPCHF10080', 'EURCHF10080', 'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]

headers = ['date', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]


In [11]:

shortest_row_currency = pd.read_csv('../recent_data/' + 'NZDCAD10080' + '.csv', names=headers  )  



#this reverses the rows so as to have the recent timeframe at the top
shortest_row_currency = shortest_row_currency.sort_index(axis=0,ascending=False)


shortest_row_date = shortest_row_currency['date'].values.tolist()


print(len(shortest_row_date))
    

728


In [12]:
missing_date = []


for x in currency_pairs:
    
    current_currency = pd.read_csv('../recent_data/' + x + '.csv', names=headers  )  
    
    current_currency_date = current_currency['date'].values.tolist()
    
    
   
   
    for j in shortest_row_date:
        
        if j not in current_currency_date:
            
            missing_date.append(j)
        
            

# this get the unique time frames because time frames could have been duplicated from different currencies 
missing_date = np.unique(missing_date)   



for x in missing_date:
    shortest_row_date.remove(x)
            
print(len(missing_date))
print(len(shortest_row_date))




combine_data = pd.DataFrame()
combine_data['date']= shortest_row_date

20
708


In [13]:
year = []
month=[]
day =[]


for c in shortest_row_date:
    year.append(c[:4] + '_year')
    month.append(c[5:7] + '_month')
    day.append(c[8:10] + '_day')

    
    
    



In [14]:

combine_data['year']= year
combine_data['month'] = month
combine_data['day'] = day

column_trans = make_column_transformer(  
    (OneHotEncoder(sparse=False), ['year','month','day'  ]),
    remainder ='passthrough')

combine_data = column_trans.fit_transform(combine_data)
combine_data.shape
column_names = column_trans.get_feature_names()
combine_data = pd.DataFrame(combine_data, columns=column_names)

print(combine_data)


    onehotencoder__x0_2007_year onehotencoder__x0_2008_year  \
0                           0.0                         0.0   
1                           0.0                         0.0   
2                           0.0                         0.0   
3                           0.0                         0.0   
4                           0.0                         0.0   
..                          ...                         ...   
703                         1.0                         0.0   
704                         1.0                         0.0   
705                         1.0                         0.0   
706                         1.0                         0.0   
707                         1.0                         0.0   

    onehotencoder__x0_2009_year onehotencoder__x0_2010_year  \
0                           0.0                         0.0   
1                           0.0                         0.0   
2                           0.0                       

In [15]:



for q in currency_pairs:

    current_currency = pd.read_csv('../recent_data/' + q + '.csv', names=headers  )
    
    
    # classify each row and attach numeric score to it
    classification_score = []
    
    
    for b in shortest_row_date:
            
        row = current_currency[current_currency.date ==  b]
        
       
        # condition when the curreny movement is a buy

        
        if row.open.item() < row.close.item():
            
            classification_score.append(1)
        else:
            classification_score.append(-1)

    current_currency_pair = q.replace('10080','')
    
    
    
    combine_data[current_currency_pair] = classification_score
           
    print(q)              

print(combine_data)              

USDCHF10080


KeyboardInterrupt: 

In [ ]:
for g in currency_pairs:
    
    new_data = combine_data.copy()
    current_currency_pair = g.replace('10080','')
    needed_column = new_data[current_currency_pair].values.tolist()
    needed_column.insert(0,0)
    needed_column.pop()
    
    new_data.insert(loc=1, column = 'nextweek_' + current_currency_pair, value= needed_column)
    new_data = new_data.iloc[1: , :]
   
   
    new_data.to_csv('files/' + current_currency_pair + '.csv', index=False)
    print(current_currency_pair)
    